# The Decision Tree on the Churn Dataset in Spark with Cross Validation

In [36]:
from pyspark.sql import DataFrameReader
from pyspark.sql import SparkSession
from pyspark.ml.feature import IndexToString, Normalizer, StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline

## Select the churn file 

In [37]:
inputFile = "../data/churn.csv"

## Create the Spark Session 

In [38]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("ChurnDecisionTree")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile)   

## Data Preparation
### Transform labels into index

In [39]:
labelIndexer = StringIndexer().setInputCol("LEAVE").setOutputCol("label").fit(df)
collegeIndexer = StringIndexer().setInputCol("COLLEGE").setOutputCol("COLLEGE_NUM").fit(df)
satIndexer = StringIndexer().setInputCol("REPORTED_SATISFACTION").setOutputCol("REPORTED_SATISFACTION_NUM").fit(df)
usageIndexer = StringIndexer().setInputCol("REPORTED_USAGE_LEVEL").setOutputCol("REPORTED_USAGE_LEVEL_NUM").fit(df)
changeIndexer = StringIndexer().setInputCol("CONSIDERING_CHANGE_OF_PLAN").setOutputCol("CONSIDERING_CHANGE_OF_PLAN_NUM").fit(df)

 ### Build the feature vector

In [40]:
featureCols = df.columns.copy()
featureCols.remove("LEAVE")
featureCols.remove("COLLEGE")
featureCols.remove("REPORTED_SATISFACTION")
featureCols.remove("REPORTED_USAGE_LEVEL")
featureCols.remove("CONSIDERING_CHANGE_OF_PLAN")
featureCols = featureCols +["COLLEGE_NUM","REPORTED_SATISFACTION_NUM","REPORTED_USAGE_LEVEL_NUM","CONSIDERING_CHANGE_OF_PLAN_NUM"]

### Build the feature Vector Assembler

In [41]:
assembler =  VectorAssembler(outputCol="features", inputCols=list(featureCols))

### Build a featureIndexer 

Automatically identify categorical features, and index them.
Features with > 5 distinct values are treated as continuous.

In [42]:
featureIndexer = VectorIndexer(inputCol="features",outputCol="indexedFeatures", maxCategories=6) 

### Convert indexed labels back to original labels

In [43]:
predConverter = IndexToString(inputCol="prediction",outputCol="predictedLabel",labels=labelIndexer.labels)

## Build the decision tree model

In [44]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

## Data Preparation
### Build a network parameters grid

In [45]:
paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [ 5, 10 ]) \
				              .addGrid(dt.minInfoGain, [0.05, 0.025, 0.02, 0.01]) \
				              .addGrid(dt.minInstancesPerNode, [5, 10, 15]) \
                              .addGrid(dt.maxBins, [5, 6, 9]) \
				              .build()

### Split the data

In [46]:
splits = df.randomSplit([0.6, 0.4 ], 1234)
train = splits[0]
test = splits[1]

### Build a pipeline

In [47]:
pipeline = Pipeline(stages= [labelIndexer, collegeIndexer, satIndexer,
				usageIndexer, changeIndexer, assembler, featureIndexer,  dt, predConverter])

## Build an evaluator

In [48]:
evaluator =  BinaryClassificationEvaluator(labelCol="label",rawPredictionCol="prediction", metricName="areaUnderROC")

## Build the Cross Validator

In [49]:
cv = CrossValidator(estimator=pipeline, evaluator=evaluator,estimatorParamMaps=paramGrid,numFolds=10, parallelism=2)

## Train the Model 

In [50]:
cvModel = cv.fit(train)

## Find out what is the best model

In [51]:
treeModel = cvModel.bestModel.stages[7]
print("Learned classification tree model:\n",treeModel)
print("Best Params: \n", treeModel.explainParams())

Learned classification tree model:
 DecisionTreeClassificationModel (uid=DecisionTreeClassifier_f4f33822ab53) of depth 10 with 27 nodes
Best Params: 
 cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featuresCol: features column name (default: features, current: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name (default: label, current: label)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categorie

## Test the model 

In [52]:
predictions = cvModel.transform(test)
predictions.select("prediction", "label", "predictedLabel", "LEAVE", "features").show()

+----------+-----+--------------+-----+--------------------+
|prediction|label|predictedLabel|LEAVE|            features|
+----------+-----+--------------+-----+--------------------+
|       0.0|  1.0|          STAY|LEAVE|[20007.0,36.0,23....|
|       1.0|  1.0|         LEAVE|LEAVE|[20009.0,183.0,18...|
|       1.0|  1.0|         LEAVE|LEAVE|[20012.0,246.0,9....|
|       1.0|  0.0|         LEAVE| STAY|[20063.0,58.0,0.0...|
|       1.0|  0.0|         LEAVE| STAY|(11,[0,3,4,6,8,10...|
|       0.0|  0.0|          STAY| STAY|[20078.0,199.0,65...|
|       0.0|  0.0|          STAY| STAY|(11,[0,3,4,6,8],[...|
|       0.0|  0.0|          STAY| STAY|[20278.0,0.0,69.0...|
|       0.0|  0.0|          STAY| STAY|[20284.0,0.0,5.0,...|
|       1.0|  1.0|         LEAVE|LEAVE|[20288.0,0.0,0.0,...|
|       0.0|  1.0|          STAY|LEAVE|[20317.0,85.0,0.0...|
|       1.0|  1.0|         LEAVE|LEAVE|[20320.0,0.0,0.0,...|
|       0.0|  1.0|          STAY|LEAVE|[20326.0,88.0,0.0...|
|       0.0|  0.0|      

## Test Error

In [53]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = " ,(1.0 - accuracy))

Test Error =  0.2959472513661693


In [54]:
spark.stop()